In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer

# Create the dataframes

In [54]:
df_train = pd.read_csv('home_depot_data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('home_depot_data/test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('home_depot_data/attributes.csv')
df_pro_desc = pd.read_csv('home_depot_data/product_descriptions.csv')

# Add stemmer

In [55]:
stemmer = SnowballStemmer('english')
def str_stemmer(s):
	return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	return sum(int(str2.find(word)>=0) for word in str1.split())

# Dataframe processing

In [56]:
df_train['search_term'] = df_train['search_term'].map(lambda x:str_stemmer(x))
df_train['product_title'] = df_train['product_title'].map(lambda x:str_stemmer(x))
df_train['len_of_query'] = df_train['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_train['product_info'] = df_train['search_term']+"\t"+df_train['product_title']
df_train['word_in_title'] = df_train['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_train = df_train.drop(['search_term','product_title', 'product_info'],axis=1)

In [57]:
df_train

,id,product_uid,relevance,len_of_query,word_in_title
0,2,100001,3.00,2,1
1,3,100001,2.50,2,1
2,9,100002,3.00,2,1
3,16,100005,2.33,3,1
4,17,100005,2.67,3,3
...,...,...,...,...,...
74062,221457,206638,1.00,3,1
74063,221458,206639,3.00,3,3
74064,221463,206641,2.33,7,2
74065,221471,206648,3.00,3,2


In [58]:
train, test = train_test_split(df_train, test_size=0.2, random_state=23)

# Define test and training sets

In [59]:
id_test = test['id']

y_train = train['relevance'].values
X_train = train.drop(['id','relevance'],axis=1).values
X_test = test.drop(['id','relevance'],axis=1).values
y_test = test['relevance'].values

In [60]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [11]:
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)